<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Vector_Magnetometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objs as go
import plotly.figure_factory as ff

In [ ]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [ ]:
wire_radius = 25e-6; distance = 50e-6; wire_current = 10; mu_freeSpace = 4*np.pi*1e-7; chi = -2.2e-5
outer_linear_density = 5; inner_linear_density = 5; point_separation = 2e-6

# outer_points = np.linspace(wire_radius, distance, outer_linear_density); inner_points = np.linspace(0,wire_radius,inner_linear_density,endpoint=False)
outer_points = np.arange(wire_radius, distance, point_separation); inner_points = np.arange(0,wire_radius,point_separation)

In [ ]:
# Customizing the number of points for a cirle
init_angle = 0; final_angle = 2*np.pi; point_distance = np.pi/4.0
init_circle_points = 1; point_increament = 2

# about_angle = np.array([init_angle,final_angle,point_distance])
about_angle_1 = np.array([init_angle,final_angle,init_circle_points,point_increament])

In [ ]:
x_linear_point = np.unique(np.concatenate((-outer_points[::-1],-inner_points[::-1][:-1],inner_points,outer_points)))
y_linear_point = x_linear_point

In [ ]:
# Important Modfications for calculating phi from (x,y)
(np.arctan(-1/-1)+np.pi)*180/np.pi,(2*np.pi+np.arctan(-1/1))*180/np.pi,(np.arctan(1/-1)+np.pi)*180/np.pi

In [ ]:
def xy_phi(x,y):
    if x==0.0:
      angle = np.arctan(y*np.inf)
    # as shown above
    elif x<0 and y<0:
      angle = np.arctan(y/x)+np.pi
    elif x>0 and y<0:
      angle = np.arctan(y/x)+2*np.pi
    elif x<0 and y>0:
      angle = np.arctan(y/x)+np.pi
    else:
      angle = np.arctan(y/x)
    return angle 

In [ ]:
def circular_grid(x,wire_radius,about_angle):
  radius  = x[np.where(x>=0)[0]]
  # init,final,separation = about_angle
  init,final,init_p,incre = about_angle
  xy_circle_grid = []
  for i in range(radius.shape[0]):
    angles = np.linspace(init,final,int(init_p),endpoint=False)
    init_p += incre
    # if radius[i]==0:
    #     angles = [0] 
    # else:
    #     angles = np.arange(init,final,(wire_radius/radius[i])*separation)
    xy_circle_grid.append(np.array([radius[i]*np.cos(angles),radius[i]*np.sin(angles)]))
  return xy_circle_grid

In [ ]:
def MagField(current,mu,chi,wire_radius,x,y):
  field = np.ones((x.shape[0],y.shape[0]))
  for i in range(x.shape[0]):
    for j in range(y.shape[0]):
      radius = np.sqrt(x[i]**2+y[j]**2)
      if radius<=wire_radius:
        field[i,j] = None
      else:
        angle=xy_phi(x[i],y[j])
        if y[j]<(-wire_radius):
          mu_below = mu*(1+chi)
          field[i,j] = (mu_below*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
        else:
          field[i,j] = (mu*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
  return field

In [ ]:
def Quiver_MagField(current,mu,chi,wire_radius,x_linear,about_angle):
  circular_field_xy = []; circular_quiver_xy = []; circular_field = []
  xy_points = circular_grid(x_linear,wire_radius,about_angle)
  for i in range(len(xy_points)):
    x = xy_points[i][0,:]; y = xy_points[i][1,:]
    _field_xy = []; _quiver_xy = []; _field = []
    for i in range(x.shape[0]):
      j=i
      radius = np.sqrt(x[i]**2+y[j]**2)
      if radius<=wire_radius:
        field = None
        field_xy = np.array([None,None])
        quiver_xy = np.array([x[i],y[j]])
        _field_xy.append(field_xy); _quiver_xy.append(quiver_xy); _field.append(field)
      else:
        angle=xy_phi(x[i],y[j])
        if y[j]<(-wire_radius):
          mu_below = mu*(1+chi)
          field = (mu_below*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
          field_xy = np.array([field*np.cos(angle), field*np.sin(angle)])
          quiver_xy = np.array([x[i],y[j]])
          _field_xy.append(field_xy); _quiver_xy.append(quiver_xy); _field.append(field)
        else:
          field = (mu*current)/(2*np.pi*np.sqrt(x[i]**2+y[j]**2))
          field_xy = np.array([field*np.cos(angle), field*np.sin(angle)])
          quiver_xy = np.array([x[i],y[j]])
          _field_xy.append(field_xy); _quiver_xy.append(quiver_xy); _field.append(field)
    circular_field_xy.append(_field_xy); circular_quiver_xy.append(_quiver_xy); circular_field.append(_field)
  return circular_field,circular_field_xy,circular_quiver_xy

In [ ]:
# circular_field,field_xy,quiver_xy = Quiver_MagField(current=wire_current,mu=mu_freeSpace,chi=chi,wire_radius=wire_radius,x_linear=x_linear_point,about_angle=about_angle)
circular_field,field_xy,quiver_xy = Quiver_MagField(current=wire_current,mu=mu_freeSpace,chi=chi,wire_radius=wire_radius,x_linear=x_linear_point,about_angle=about_angle_1)
mag_field = MagField(current=wire_current,mu=mu_freeSpace,chi=chi,wire_radius=wire_radius,x=x_linear_point,y=y_linear_point)

In [ ]:
fig = go.Figure()
# Heatmap
fig.add_trace(go.Contour(x=x_linear_point,y=y_linear_point,z=mag_field,contours_coloring='heatmap',
                                connectgaps = True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right'),colorscale=None))
fig.update_traces(line_width=0)

# Quiver
for i in range(0,len(quiver_xy),2):
    for j in range(len(quiver_xy[i])):
        x = quiver_xy[i][j][0]; y = quiver_xy[i][j][1]
        radius = np.sqrt(x**2+y**2)
        if np.round(radius,8)>wire_radius:
            # print(i,j,radius)
            _x = [x]; _y = [y]
            u = [-field_xy[i][j][1]]; v =[ field_xy[i][j][0]]
            quiver_trace = ff.create_quiver(_x, _y, u, v, scale=0.00006, arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black')).data[0]
            fig.add_trace(quiver_trace)

# Inner Circle
fig.add_trace(go.Scatter(x=wire_radius*np.cos(np.linspace(0,2*np.pi,1000,endpoint=False)), y=wire_radius*np.sin(np.linspace(0,2*np.pi,1000,endpoint=False)),
                         mode='lines', line=dict(color='yellow', width=0.7),fill = 'tozeroy',fillcolor = 'rgba(127, 0, 255, 0.4)'))

# # Outer Circles
# [fig.add_trace(go.Scatter(x=radius*np.cos(np.linspace(0,2*np.pi,1000)), y=radius*np.sin(np.linspace(0,2*np.pi,1000)),
#                          mode='lines', line=dict(color='yellow', width=0.7))) for radius in outer_points]

# Other Ornamentations
fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker_size=15, marker=dict(color='yellow')))
fig.add_trace(go.Scatter(x=x_linear_point, y=-wire_radius*np.ones(x_linear_point.shape[0]), mode='lines', marker=dict(color='yellow'),fill=None))
fig.add_trace(go.Scatter(x=x_linear_point, y=-y_linear_point[-1]*np.ones(x_linear_point.shape[0]), mode='lines', marker=dict(color='grey'),
                         fill='tonexty',fillcolor = 'rgba(255, 255, 255, 0.5)'))

fig.update_layout(template=fig_template, title = None,width=800, height=700)
fig.update_xaxes(title = 'Distance',range = [x_linear_point[0],x_linear_point[-1]])
fig.update_yaxes(title = 'Distance', range = [y_linear_point[0],y_linear_point[-1]])
fig.update_layout(showlegend=False)
fig.show()